In [2]:
import pandas as pd
import os
import glob
import json
import random
import logging
import time
from multiprocessing import Pool

In [141]:
logger = logging.getLogger('locations_log')
logger.setLevel(logging.DEBUG)
fh = logging.FileHandler('locations.log', mode='w')
fh.setLevel(logging.DEBUG)
logger.addHandler(fh)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(process)d - %(message)s')
fh.setFormatter(formatter)
logger.addHandler(fh)

In [3]:
origin = "data/devices_json"
files = os.listdir(origin)
files[:5]

['FILE7424afef-22c4-42c8-8a6c-9fa537499329.json',
 'FILEa7e94e8b-c6b9-4cb2-a5ff-a509f1205d4d.json',
 'FILE7709cff9-43b8-4568-ba0d-e61ca70a94d0.json',
 'FILE3a6714f7-9b15-477f-b04b-b3d9fed26fe9.json',
 'FILE8036789b-87a9-4b6f-be1b-e16880f02f1a.json']

In [4]:
res = ['7424afef-22c4-42c8-8a6c-9fa537499329', 'a7e94e8b-c6b9-4cb2-a5ff-a509f1205d4d']
res

['7424afef-22c4-42c8-8a6c-9fa537499329',
 'a7e94e8b-c6b9-4cb2-a5ff-a509f1205d4d']

In [17]:
res = db.locations.distinct("userid")
toprocess = list(set(files) - set(["FILE{}.json".format(x) for x in res]))

In [155]:
tmp = pd.DataFrame(files, columns=['json'])
tmp['search'] = tmp.apply(lambda x: '-6889-44af-b846-96284ee3619c' in x['json'], axis=1)

In [152]:
tmp['search'].sum()

0

In [156]:
tmp.shape

(526213, 2)

In [143]:
def locationsJson(file):
    ts = time.time()
#     file = files[0]
    filepath = os.path.join(origin, file) 

    with open(filepath , 'r') as fp:
        record = fp.read()
        record = json.loads(record)

    # print(type(record))
    # record
    df = pd.DataFrame.from_dict(record['location'], dtype=int)
    df['_id'] = record['_id']
    df = df[['_id','index', 'date', 'lat', 'lon', 'label', 'h3_10', 'h3_9', 'h3_8', 'h3_7', 'h3_6']]
    df['index'] = df['index'].astype(float)
    df['label'] = df['label'].astype(float)
    # df.head()
    _id = df['_id'][0]
    
    for i in df.index:
        name = "file_{}_loc_{}.json".format(_id, int(i)) #file-location.json
        fileJsonPath = os.path.join(path, name)
        record = df.loc[i].to_dict()
    #     print(fileJsonPath)
        with open(fileJsonPath, 'w') as fp:
    #         fp.write(json.dumps(record))
            json.dump(record, fp, indent=3)

    logger.info('| %s | %s | %s ', file, df.shape, time.time() - ts)

In [144]:
if __name__ == '__main__':
    ts = time.time()
    pool = Pool(os.cpu_count())                         # Create a multiprocessing Pool

    origin = "data/devices_json"
    files = os.listdir(origin)
    print(len(files))
#     files = files[:10]
    logger.info("Total files: {}".format(len(files)))
    pool.map(locationsJson, files)

    logger.info('Process finished in {} seconds'.format(time.time() - ts))

526213


OSError: [Errno 28] No space left on device: 'data/locations_json/file_a89fbda2-34ed-42b2-ad36-8c82def53a5a_loc_6634.json'

In [134]:
df['_id'][0]

'7424afef-22c4-42c8-8a6c-9fa537499329'

In [99]:
path = "data/locations_json/"
_id = '7424afef-22c4-42c8-8a6c-9fa537499329'

# file = jsonFiles[0]
# with open(file , 'r') as fp:
#     record = fp.read()
#     record = json.loads(record)

for i in df.index:
    name = "file_{}_loc_{}.json".format(_id, i)
    fileJsonPath = os.path.join(path, name)
    record = df.loc[i].to_dict()
#     print(fileJsonPath)
    with open(fileJsonPath, 'w') as fp:
        fp.write(json.dumps(record))



TypeError: Object of type 'int64' is not JSON serializable